In [53]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
import math
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio

<!-- ##### Object for individual customer
- Calculates net-load of each customer. Inputs: each customer's demand, microgeneration forecasts, initial SOC, battery status (0/1)
- Energy management at the household -->

In [ ]:
class Customer: # Customer Energy Management
    def __init__(self,hh_demand,pvsize,hh_SOC,batt_size,Pr_grid):
        self.hh_demand=hh_demand
        self.hh_gen=Pg.iloc[t].values*pvsize
        self.hh_SOC=hh_SOC
        self.batt_size=batt_size
        # self.Pr_fit=fit
        self.Pr_grid=Pr_grid
        self.E=batt_size
        self.Pbmax=batt_size*Crate

        self.hh_demand = self.price_responsive_load()

        if self.batt_size>0: #battery management
            self.battery=self.battery_management()
            self.Pgrid=self.battery[0]
            self.SOC2=self.battery[1]
            self.Pb=self.battery[2]
        else: #Energy management in absence of battery
            self.Pgrid=(self.hh_demand-self.hh_gen)
            self.SOC2=0
            self.Pb=0

    def price_responsive_load(self):
        if self.Pr_grid >= Pr_th_s:
            self.hh_demand = 0.5 * self.hh_demand + 0.5 * self.hh_demand * math.exp(-8 * (Pr_th_s - Pr_th_b))
        elif self.Pr_grid >= Pr_th_b and self.Pr_grid <= Pr_th_s:
            self.hh_demand = 0.5 * self.hh_demand + 0.5 * self.hh_demand * math.exp(-8 * (self.Pr_grid - Pr_th_b))
        else:
            self.hh_demand = 0.5 * self.hh_demand + 0.5 * self.hh_demand * math.exp(-8 * (Pr_th_b - Pr_th_b))
        return self.hh_demand

    def battery_management(self):
        if self.hh_gen>self.hh_demand:
            if self.Pr_grid>=Pr_th_s: #sell from the battery and the surplus gen 
                self.Pbcal=(self.hh_SOC-SOC_min)*(self.E*nd)/dT #battery discharging
                if self.Pbcal>=self.Pbmax:
                    self.Pb=-self.Pbmax
                else:
                    self.Pb=-self.Pbcal
            elif self.Pr_grid>Pr_th_b: #dead band
                self.Pb=0
            else:
                self.Pbcal=(SOC_max-self.hh_SOC)*self.E*nc/dT #battery charging if price < buying threshold
                if self.Pbcal>=self.Pbmax:
                    self.Pb=self.Pbmax
                else:
                    self.Pb=self.Pbcal
        else:
            if self.Pr_grid<=Pr_th_b:
                self.Pbcal=(SOC_max-self.hh_SOC)*self.E/(nc*dT) #battery charging
                if self.Pbcal>=self.Pbmax:
                    self.Pb=self.Pbmax
                else:
                    self.Pb=self.Pbcal
            elif self.Pr_grid<=Pr_th_s:
                self.Pb=0
            else:
                self.Pbcal=(self.hh_SOC-SOC_min)*(self.E*nd)/dT #discharging
                if self.Pbcal>=self.Pbmax:
                    self.Pb=-self.Pbmax
                else:
                    self.Pb=-self.Pbcal
        self.Pgrid=(self.hh_demand-self.hh_gen)+self.Pb
        self.SOCend=self.hh_SOC + ((self.Pb*nc)*(dT/self.E) if self.Pb>=0 else (self.Pb/nd)*(dT/self.E))
        return self.Pgrid, self.SOCend, self.Pb

In [55]:
class CustomerOffgrid:
    def __init__(self,hh_demand,pvsize,hh_SOC,batt_size):
        self.hh_demand=hh_demand
        self.hh_gen=Pg.iloc[t].values*pvsize
        self.hh_SOC=hh_SOC
        self.batt_size=batt_size
        self.E=batt_size
        self.Pbmax=batt_size*Crate
        self.battery=self.battery_management()
        self.Pb=self.battery[0]
        self.SOC2=self.battery[1]
        self.Pcurtail=self.battery[2]
        
    def battery_management(self):
        if self.hh_gen>self.hh_demand:
            self.Pbcal=(SOC_max-self.hh_SOC)*self.E/(nc*dT) #Charging
            if self.Pbcal>=self.Pbmax:
                self.Pbreq=self.Pbmax
            else:
                self.Pbreq=self.Pbcal
            if (self.hh_gen-self.hh_demand)>=self.Pbreq:
                self.Pb=self.Pbreq 
            else:
                self.Pb=(self.hh_gen-self.hh_demand)
        else:
            self.Pbcal=(self.hh_SOC-SOC_min)*(nd*self.E)/dT
            if self.Pbcal>=self.Pbmax:
                self.Pbav=self.Pbmax
            else:
                self.Pbav=self.Pbcal
            if (self.hh_demand-self.hh_gen)>=self.Pbav:
                self.Pb=-self.Pbav
            else:
                self.Pb=-(self.hh_demand-self.hh_gen) #discharge is -ve Pb
        self.Pcurtail=self.hh_demand-self.hh_gen+self.Pb
        if self.batt_size>0:
            self.SOCend=self.hh_SOC + ((self.Pb*nc)*(dT/self.E) if self.Pb>=0 else (self.Pb/nd)*(dT/self.E))
        else:
            self.SOCend=0
        return self.Pb, self.SOCend, self.Pcurtail

In [56]:
class LCOEoff:
    def __init__(self,SystemCost):
        self.SystemCost=SystemCost
        self.lcoeoff=self.calculate_lcoeoff()
        
    def calculate_lcoeoff(self):
        self.lcoeoff=((self.SystemCost) + (Inv[k-1]*10/15 if k>=1 else 0) + (Inv[k-2]*5/15 if k>=2 else 0))/(Pd.iloc[h,:].sum()*15)
        return self.lcoeoff

##### Object to calculate hourly bill of individual customer

In [57]:
class Bill:
    def __init__(self,demand,R_VC,R_FC,T_C,D_C,M_C,D_FC,h):
        self.demand=demand
        self.Retailer_VC=R_VC
        self.Retailer_FC=R_FC
        self.T_cost=T_C #transmission costs per kWh
        self.D_cost=D_C #distribution costs per kWh
        self.M_cost=M_C #municipal costs per kWh
        self.dist_fixed_cost_HH=D_FC
        self.electricity_charges=self.calculate_electricity_charges()
        self.energy_credits=self.calculate_energy_credits()
        self.delivery_charges=self.calculate_delivery_charges()
        self.electricity_bill=self.calculate_electricity_bill()

    def calculate_electricity_charges(self):
        evarcharge=np.sum(self.demand * self.Retailer_VC)
        efixedcharge=len(self.demand)*self.Retailer_FC
        echarge=evarcharge+efixedcharge
        return echarge
    
    def calculate_energy_credits(self):
        ecredit=np.sum(np.where(self.demand < 0, -self.demand * self.Retailer_VC, 0))
        return ecredit
    
    def calculate_delivery_charges(self):
        dcharge1=[dd*(self.T_cost+self.D_cost+self.M_cost+Other_ch) if dd>0 else 0 for dd in self.demand]
        dcharge2=len(self.demand)*(self.dist_fixed_cost_HH+rate_riders)
        dcharge=(sum(dcharge1)+dcharge2)
        return dcharge
    
    def calculate_electricity_bill(self):
        bill=(self.electricity_charges+self.Retailer_FC+self.delivery_charges)
        return bill

In [58]:
class Systemsize:
    def __init__(self,total_energy_consumption):
        self.total_energy_consumption=total_energy_consumption
        self.maxlimitpv=1
        panelsize=0.4 #kW
        # PVCost=3064.56 #$/kW
        annual_om_cost_per_kw=OnM.iloc[k*5]
        BattCost=0.6*Batteryadvanced.iloc[k*5]
        PVCost=PVadvanced.iloc[k*5]
        # BattCost=654 #$/kW

        X2=self.maxlimitpv*self.total_energy_consumption/(solar_prod_annual)
        X=X2-(pvk[k-1,h] if k>=1 else 0)-(pvk[k-2,h] if k>=2 else 0)
        X1=math.ceil(X/panelsize) #number of 0.4 kW PV panels
        # print("AEC/AEP per kW = ", X, "Number of 0.4kW Panels = ", X1)

        #PV size options: 100%, 50%, 25%, 0
        PV1=X1*panelsize 
        PV2=math.floor(X1/2)*panelsize
        PV3= math.floor(X1/4)*panelsize
        PV4=0
        data_PV = {'PVSize': [PV1, PV2, PV3, PV4],
                  'PVCost': [PVCost*PV1, PVCost*PV2, PVCost*PV3, PVCost*PV4]}
        df_PV = pd.DataFrame(data_PV)
        
        #Battery size options: round down values, closest to selected PV sizes
        B1=math.floor(PV1)/Crate
        B2=math.floor(PV2)/Crate
        B3=math.floor(PV3)/Crate
        B4=math.floor(PV4)/Crate
        # print('1:', 'PV=', PV1, "Batt=", B1, '2:', 'PV=', PV2, "Batt=",  B2, '3:', 'PV=', PV3, "Batt=", B3, '4:', 'PV=', PV4, "Batt=", B4)
        data_B = {'BattSize': [B1, B2, B3, B4],
                  # 'BattSize': [B1, B2, B3, B4],
                  # 'BattCost': [BattCost*B1, BattCost*B2, BattCost*B3, 0]} #price of battery
                  'BattCost': [BattCost*B1, BattCost*B2, BattCost*B3, 0]} #price of battery
        df_B = pd.DataFrame(data_B)  
       
        self.df_combinations = pd.DataFrame(columns=['PVSize', 'BattSize', 'Total_Cost']) 

        for _, row_PV in df_PV.iterrows():
            for _, row_B in df_B.iterrows():
                total_cost = row_PV['PVCost'] + row_B['BattCost'] + annual_om_cost_per_kw*row_PV['PVSize']*systemlifetime
                data = [[row_PV['PVSize'], row_B['BattSize'], total_cost]]
                self.df_combinations = pd.concat([self.df_combinations, pd.DataFrame(data, columns=['PVSize', 'BattSize', 'Total_Cost'])])

        self.df_combinations.reset_index(drop=True, inplace=True)  # Reset the index
        print(self.df_combinations)

##### Object for individual customer's decision

In [59]:
class Decision:
    # def __init__(self,Electricity_Bill,PVSize,PVSize0,BattSize,BattSize0,BillPV0,SystemCost,BaseBilla,BaseLoada,Loff):
    def __init__(self,Electricity_Bill,PVSizedecision,BattSizedecision,SystemCost,Loff):
        self.electricity_bill=Electricity_Bill
        self.PVSizedecision=PVSizedecision
        self.BattSizedecision=BattSizedecision
        self.SystemCost=SystemCost
        self.options=options
        self.Loff=Loff
        self.LCOE=np.zeros(self.options)
        self.lcoge=self.calculate_lcoge() #levelized cost of grid electricity
        self.PV_lcoe=self.calculate_PV_lcoe() #levelized cost of each PV-batt combination
        self.system_decision=self.select_system() #logic to select one combination

    def calculate_lcoge(self): #previous investment?
        self.lcoge = round((Inv[k-2,h]*5/15 + Inv[k-1,h]*10/15 + Billexpected[k,h]*5 + Billkminus1k[h,15]*5 + Bill0[k,h]*5)/(Pd.iloc[h,:].sum()*15) if k>=2 else (Inv[k-1,h]*10/15 + Billexpected[k,h]*10 + Bill0[k,h]*5)/(Pd.iloc[h,:].sum()*15) if k==1 else (Bill0[k,h]*15)/(Pd.iloc[h,:].sum()*15),3)
        print('grid lcoe',self.lcoge)
        return self.lcoge
    
    def calculate_PV_lcoe(self):
        for i in range(0,self.options):
            self.LCOE[i]=round((Inv[k-2,h]*5/15+Inv[k-1,h]*10/15+self.SystemCost[i]+self.electricity_bill[i]*5+Billkminus1k[h,i]*5+Billonlyk[h,i]*5)/(Pd.iloc[h,:].sum()*15) if (k-2) >= 0 else (Inv[k-1,h]*10/15+self.SystemCost[i]+self.electricity_bill[i]*10+Billonlyk[h,i]*5)/(Pd.iloc[h,:].sum()*15) if k==1 else (self.SystemCost[i] + self.electricity_bill[i]*15)/(Pd.iloc[h,:].sum()*15),3)
        return(self.LCOE)
    
    def select_system(self):
        print('LCOE combinations',self.LCOE)
        self.L= np.append(self.LCOE, self.Loff)
        self.PV_add_lcoe=min(filter(lambda y: y <= self.lcoge, self.L), default=None)
        index = np.where(self.L==self.PV_add_lcoe)[0]

        if len(index) == 0:
            print('No index found for selected LCOE.')
            self.Selected_PV = 0
            self.SelectedBatt = 0
            self.index = -1  # Define index to avoid AttributeError
            return 0

        self.index = index[0]
        self.Selected_PV=self.PVSizedecision[self.index]
        self.SelectedBatt=self.BattSizedecision[self.index]
        print('selected',self.L[self.index])
        print('selected PV size', self.Selected_PV)
        print('selected Batt size', self.SelectedBatt)
        return self.PV_add_lcoe

##### Object for Retailer Cost Calculation- BaU

In [60]:
#Business-as-usual
#Object for retailer energy-related and fixed-costs
class Energy_costs: 
    def __init__(self,TotalLoad1,cont_vol,cont_price,WS_price,ret_margin_E,ret_margin_NE,risk_margin,admin_ch,fees,HourlyResLoad,hourlynet,hourly_load,hourly_ugen,connectedhouses,dvc):
        #Energy Procurement variables
        self.cont_vol=cont_vol #contract volume
        self.cont_price=cont_price #contract price
        self.total_load=TotalLoad1
        #Variable cost components
        self.ret_margin_E=ret_margin_E #energy return margin
        self.ret_margin_NE=ret_margin_NE #return margin on non-energy costs
        self.risk_margin=risk_margin
        #Fixed cost components
        self.hourlynetload=hourlynet
        self.hourly_load=hourly_load
        self.hourly_ugen=hourly_ugen
        self.no_of_HH=connectedhouses
        self.resload=HourlyResLoad
        self.dvc=dvc

        # self.WS_vol=self.calculate_WS_vol()
        self.energy_proc_cost=self.calculate_energy_proc_cost()
        # self.Ret_Var_Cost=self.calculate_Ret_Var_Cost()
        self.Ret_Var_Cost_pu=self.calculate_Ret_Var_Cost_pu()
        # # self.FiT=self.calculate_FiT()
        self.ret_fixed_cost=self.calculate_ret_fixed_cost()
        self.ret_fixed_cost_HH=self.calculate_ret_fixed_cost_HH()
        self.ret_ugen_payment=self.calculate_ret_ugen_payment()
        # self.ret_rev=self.calculate_ret_rev()
        self.ret_rev=self.calculate_ret_rev()
        self.dsocharges=self.calculate_chargestoDSO()

 
    def calculate_energy_proc_cost(self): #energy procurment costs from contracts and WS markets
        epcost=(WS_price*np.maximum(0,(self.hourlynetload-cont_vol))+cont_vol*cont_price).sum(axis=1)
        print('market cost', (WS_price*np.maximum(0,(self.hourlynetload-cont_vol))).values.sum())
        print('cont cost', (cont_vol*cont_price).values.sum())               
        return epcost
    
    def calculate_Ret_Var_Cost_pu(self): #retailer's total variable costs
        print('EP cost', (self.energy_proc_cost.sum())*(1+self.ret_margin_E+self.risk_margin))
        print('load', (self.hourlynetload.values.sum()))
        self.rvc=((self.energy_proc_cost)*(1+self.ret_margin_E+self.risk_margin)/(self.hourly_load.sum(axis=1)))
        # self.rvc=((self.energy_proc_cost)*(1+self.ret_margin_E+self.risk_margin)/((np.maximum(0,(self.hourly_load-cont_vol)))).sum(axis=1))
        return self.rvc
    
    def calculate_ret_fixed_cost(self): #retailer's total fixed costs per hour over 15 years
        return (admin_ch)*(1+ret_margin_NE)
      
    def calculate_ret_fixed_cost_HH(self): #fixed costs per household: per hour charge
        return (self.ret_fixed_cost/((5*households.sum(axis=1)*8760)))
    
    def calculate_ret_ugen_payment(self): #microgeneration payment by retailer @ FiT Negative Cost
        ugenpay=(self.Ret_Var_Cost_pu*self.hourly_ugen.sum(axis=1)).sum()
        return ugenpay
    
    # def calculate_ret_rev(self):
    #     # print('self.ret_fixed_cost',self.ret_fixed_cost)
    #     return ((self.Ret_Var_Cost_pu*self.hourly_load.sum(axis=1)).sum()+self.ret_fixed_cost)
    
    def calculate_ret_rev(self):
         return ((self.Ret_Var_Cost_pu*base.sum(axis=1)*1000).sum())
        
    def calculate_chargestoDSO(self):
        dsocharges=((self.dvc)*self.resload.sum(axis=1)).sum()+Fixed_connection_charges
        return dsocharges

##### Object for Delivery Cost Calculation

In [61]:
class Delivery_Costs:
    def __init__(self,total_load,h):
        #margin included in these costs
        self.Fixed_connection_charges=Fixed_connection_charges #fixed distribution connection charges
        self.total_load=total_load
        self.no_of_HH=h
        self.T_cost=self.calculate_T_cost()
        self.D_cost=self.calculate_D_cost()
        self.M_cost=self.calculate_M_cost()
        self.dist_var_cost_pu=self.calculate_dist_var_cost_pu()
        self.dist_fixed_cost_HH=self.calculate_dist_fixed_cost_HH()

    def calculate_T_cost(self):
        return Tcost
    # /self.total_load
    
    def calculate_D_cost(self):
        return Dcost
    # /self.total_load
    
    def calculate_M_cost(self):
        return Mcost
    # /self.total_load
    
    def calculate_dist_var_cost_pu(self): #variable costs per kWh
        return (Tcost+Dcost+Mcost)
    # *self.total_load
    
    def calculate_dist_fixed_cost_HH(self): #fixed costs per house per hour
        return (self.Fixed_connection_charges/(5*households.values.sum()*8760))

In [62]:
kmax=3
houses=5
totalhouses=321900.6
yy=0
yyend=15
cagr=0.016
cagrhouses=0.02
planninghorizon=yyend-yy
hours=8760
cont_price=0.15
ret_margin_E=0.15
ret_margin_NE=0.05
risk_margin=0.05
Tcost=0.042094
Dcost=0.013033
Mcost=0.017162
# Tcost=0.042094*5.5e+10*planninghorizon
# Dcost=0.013033*5.5e+10*planninghorizon
# Mcost=0.017162*5.5e+10*planninghorizon
Fixed_connection_charges=0.647903*365*planninghorizon*totalhouses*5
Other_ch=0
# 0.002252 #$/kWh balance pool allocation
rate_riders=0 #enter the values as $/h 
admin_ch=0.2392*totalhouses*5*365*planninghorizon
fees=0
SOC_min=0.10
SOC_max=1
Crate=1
# 0.25
Pr_th_s=0.16
Pr_th_b=0.14
dT=1
nc=0.95
nd=0.95
systemlifetime=15 #system lifetime in years
solar_prod_annual=1167   #kWh/yr
OnM=pd.read_csv('o&mcost.csv', delimiter=',', header=None)
Pg = pd.read_csv('PV.csv', delimiter=',', header=None)
contractsize = pd.read_csv('cont_vol.csv', delimiter=',', header=None)
WSprice = pd.read_csv('WSprice.csv',usecols=[1], delimiter=',', header=None)
S2 = pd.read_csv('soc.csv', delimiter=',', header=None) #5000 houses
Pd = pd.read_csv('house_load_selected.csv',delimiter=',', header=None) #5 houses
Batteryadvanced= pd.read_csv('battcost1.csv', delimiter=',', header=None)
PVadvanced=pd.read_csv('pvcost1.csv', delimiter=',', header=None)
baseload = pd.read_csv('baseload.csv',delimiter=',', header=None) #1 year

In [63]:
Retailer_VC=np.zeros((kmax+1,hours))
Retailer_FC=np.zeros(kmax)
T_cost=np.zeros(kmax)
D_cost=np.zeros(kmax)
M_cost=np.zeros(kmax)
D_Fcost=np.zeros(kmax)

DSO_charges=np.zeros((kmax))
DSO_charges_actual=np.zeros((kmax))
Retailer_VC_actual=np.zeros((kmax+1,hours))
Retailer_FC_actual=np.zeros(kmax)
Retailer_Rev=np.zeros((kmax))
Retailer_Rev_actual=np.zeros((kmax))
Retailer_Revenue_Household=np.zeros((kmax))
Retailer_Revenue_Household_actual=np.zeros((kmax))
profit=np.zeros(kmax)
actualprofit=np.zeros(kmax)


energycost=np.zeros(kmax)
energycostactual=np.zeros(kmax)

T_cost_actual=np.zeros(kmax)
D_cost_actual=np.zeros(kmax)
M_cost_actual=np.zeros(kmax)
D_Fcost_actual=np.zeros(kmax)
st=np.zeros(houses)

ugenpayment=np.zeros(kmax)
ugenpaymentactual=np.zeros(kmax)
Electricity_Bill=np.zeros((16,houses))
Electricity_Bill_actual=np.zeros((kmax,houses))
Actualdcharge=np.zeros((kmax,houses))
Actualbill=np.zeros((kmax,houses))
Actualbillkminus1k=np.zeros((kmax,houses))
Actualbillonlyk=np.zeros((kmax,houses))
Actualbill0=np.zeros((kmax,houses))

Bill0=np.zeros((kmax,houses))

Credits=np.zeros((kmax,houses))
Credits_actual=np.zeros((kmax,houses))
Demand=np.zeros((houses,hours))
Base_demand=np.zeros((hours))
EC2=0
EG2=0
EB2=0
hh=0
system_size_b = pd.DataFrame(0, index=range(houses), columns=[""])
batt=np.zeros(houses)

battk=np.zeros((kmax,houses))
pvk=np.zeros((kmax,houses))

s=pd.DataFrame(kmax, index=range(houses), columns=[""])
Inv=np.zeros((kmax,houses))
flags = [False] * houses

Billexpected=np.zeros((kmax,houses))

In [64]:
for k in range(0,kmax):
    print('k=', k)
    base=pd.DataFrame()
    Phour=pd.DataFrame()
    Pinj=pd.DataFrame()
    cont_vol = pd.DataFrame()
    WS_price = pd.DataFrame()
    households = pd.DataFrame()
    Phour2=pd.DataFrame()
    Pinj2=pd.DataFrame()
    
    for y in range(yy,yyend):
        ind=y-yy
        base[ind] = baseload*(1+cagr)**(y)
    
    for y in range(yy,yyend):
        ind=y-yy
        households[ind] = [totalhouses*(1+cagrhouses)**(y)]
        
    WS_price1=(WSprice[(0*hours):(hours*15)]/1000)
    WS_price = pd.DataFrame()
    values_per_column = 8760
    num_columns = 15
    for i in range(num_columns):
        start_idx = i * values_per_column
        end_idx = (i + 1) * values_per_column
        WS_price[i] = WS_price1[start_idx:end_idx].reset_index(drop=True)
         
    lcoeoffgrid = np.zeros(houses)
    PV = np.zeros(houses)
    Battery = np.zeros(houses)
    
    CustomerLoad0 = pd.DataFrame()
    CustomerLoad1 = pd.DataFrame()
    CustomerBatt1 = pd.DataFrame()
    CustomerSOC1 = pd.DataFrame()
    
    Customerloadactual = np.zeros((hours,houses))
    Customersocactual = np.zeros((hours+1,houses))
    Customerbattactual = np.zeros((hours,houses))
    CustomerLoadPVonlykactual = np.zeros((hours,houses))
    Loadkminus1kactual = np.zeros((hours,houses))

    ECurtail = np.zeros((houses,hours))
    CustomerSOCOff = np.zeros((houses,hours+1))
    CustomerbattOff = np.zeros((houses,hours))

    BillPVCombinations = np.zeros((houses,16))
    Billonlyk = np.zeros((houses,16))
    Billkminus1k = np.zeros((houses,16))
   
    Dcharge = np.zeros((houses,16))
    Echarge = np.zeros((houses,16))
    
    #EM with initial PV and Battery (Bill(k-1,k-2)) to get historic load estimate for retailer and DSO
    for h in range(0,houses):
        loads1 = []
        batp1 = []
        soc1 = []
        PV[h]=(pvk[k-2,h] if k>=2 else 0) + (pvk[k-1,h] if k>=1 else 0)
        Battery[h]=(battk[k-2,h] if k>=2 else 0) + (battk[k-1,h] if k>=1 else 0)
        SOC=SOC_max if Battery[h]>0 else 0
        soc1.append(SOC)
        for t in range(0,hours): #8760
            instance1 = Customer(Pd.iloc[h,t], PV[h], SOC, Battery[h], Retailer_VC[k,t])
            SOC=instance1.SOC2
            loads1.append(instance1.Pgrid)
            batp1.append(instance1.Pb)
            soc1.append(instance1.SOC2)
        column_name = f'House {h}'
        
        CustomerLoad1[column_name] = loads1
        CustomerBatt1[column_name] = batp1
    
    Load1 = np.where(CustomerLoad1 >= 0, CustomerLoad1, 0)
    Inj1 = np.where(CustomerLoad1 < 0, CustomerLoad1, 0)
    TotalLoad_h = Load1.sum() #kWh
    TotalInj_h = Inj1.sum() #kWh
    HourlyLoad1_h = (Load1.sum(axis=1)) #kWh
    HourlyInj1_h = Inj1.sum(axis=1)
    HourlyLoad0 = pd.DataFrame(HourlyLoad1_h)
    HourlyInj0 = pd.DataFrame(HourlyInj1_h)
    # HourlyLoad1 = (HourlyLoad0*321900.6)+(base*1000) #kWh
    # HourlyInj1 = HourlyInj0*321900.6
    # cont_vol=contractsize[0]*1000

    for y in range(yy,yyend):
        ind=y-yy
        cont_vol[ind]=contractsize[0]*1000 #kWh

    for y in range(yy,yyend):
        ind=y-yy
        Phour[ind] = HourlyLoad0[0]*households.iloc[0,ind]
    HourlyLoad1=(Phour)+(base*1000) #kWh
    HourlyResLoad1=Phour

    for y in range(yy,yyend):
        ind=y-yy
        Pinj[ind] = HourlyInj0[0]*households.iloc[0,ind]
    HourlyInj1 = (Pinj) #kWh
    
    HourlyNet1=HourlyLoad1+HourlyInj1

    TotalLoad1=(TotalLoad_h*households.values.sum())+(base.values.sum()*1000) #kWh
    print('TotalLoad_h',TotalLoad_h)
    print('households.values.sum()', households.values.sum())
    print('(base.values.sum()*1000)',(base.values.sum()*1000))
    Totalresload=(TotalLoad_h*households.values.sum()) #kWh
    TotalInj1=(TotalInj_h*households.values.sum()) #kWh
    TotalNet1=TotalLoad1+TotalInj1

    print('Reductions in houseshold load', (Pd.sum()-HourlyLoad0[0]).sum()*households.values.sum())
    a=base.sum()*1000
    print('Increased base load', (a-a[0]).sum())
    # print('Increased house load', )
    print('Total Grid Load for 15 years:', TotalLoad1, 'kWh')
    print('Total Res Load for 15 years:', Totalresload, 'kWh')
    print('Total Injection for 15 years:', TotalInj1, 'kWh')


    Distribution=Delivery_Costs(Totalresload,houses-hh)
    Retailer = Energy_costs(TotalNet1,cont_vol,cont_price,WS_price,ret_margin_E,ret_margin_NE,risk_margin,admin_ch,fees,HourlyResLoad1,HourlyNet1,HourlyLoad1,HourlyInj1,houses-hh,Distribution.dist_var_cost_pu)
    # Distribution=Delivery_Costs(TotalLoad1,houses-hh)
    # Distribution=Delivery_Costs(Totalresload,houses-hh)
    energycost[k]=Retailer.energy_proc_cost.sum()
    Retailer_VC[k]=Retailer.Ret_Var_Cost_pu
    Retailer_FC[k]=Retailer.ret_fixed_cost_HH
    Retailer_Rev[k]=Retailer.ret_rev
    ugenpayment[k]=Retailer.ret_ugen_payment #negative value
    # profit[k]=Retailer_Rev[k]+ugenpayment[k]-energycost[k]
    T_cost[k]=Distribution.T_cost
    D_cost[k]=Distribution.D_cost
    M_cost[k]=Distribution.M_cost
    D_Fcost[k]=Distribution.dist_fixed_cost_HH
    DSO_charges[k]=Retailer.dsocharges
    print('Total Energy Cost',energycost[k] ,'C$')
    print("Retailer VC", Retailer_VC[k], 'C$/kWh')
    print("Retailer FC", Retailer_FC[k], 'C$/h')
    print("Transmission cost", T_cost[k], 'C$/kWh')
    print("Distribution cost", D_cost[k], 'C$/kWh')
    print("Municipal cost", M_cost[k], 'C$/kWh')
    # print("Retailer revenue", Retailer_Rev[k], 'C$')
    print('Payment by Retailer for Injections', ugenpayment[k], 'C$')
    # print('Retailer Profit', profit[k], 'C$')
    # print('households',households)
    
#     # colors = px.colors.qualitative.Plotly
#     # fig = go.Figure()
#     # fig.add_traces(go.Scatter(y = HourlyLoad1, mode = 'lines', line=dict(color=colors[0]), name='Load'))
#     # fig.add_traces(go.Scatter(y = Retailer_VC[k], mode = 'lines', line=dict(color=colors[0]), name='Retail Price'))
#     # fig.update_layout(title='Base')
#     # # pio.write_html(fig, 'Offgrid.html')
#     # fig.show()
    
    for h in range(0,houses):
        billexp = Bill(CustomerLoad1.iloc[:,h],Retailer_VC[k],Retailer_FC[k],T_cost[k],D_cost[k],M_cost[k],D_Fcost[k],h)
        Billexpected[k,h] = billexp.electricity_bill
    Retailer_Revenue_Household[k]=Billexpected[k].sum()*(households.sum(axis=1))
    profit[k]=Retailer_Revenue_Household[k]+Retailer_Rev[k]-energycost[k]-DSO_charges[k]+ugenpayment[k]-admin_ch
    print("Retailer revenue total", Retailer_Revenue_Household[k]+Retailer_Rev[k], 'C$')
    print('Cost to DSO', DSO_charges[k], 'C$')
    print('Retailer Profit', profit[k], 'C$')

    
    for h in range(0,houses):
        # billexp = Bill(CustomerLoad1.iloc[:,h],Retailer_VC[k],Retailer_FC[k],T_cost[k],D_cost[k],M_cost[k],D_Fcost[k],h)
        # Billexpected[k,h] = billexp.electricity_bill
        loads0 = []
        SOC0=0
        batt0=0
        for t in range(0,hours):
            Generation0=0
            instance0 = Customer(Pd.iloc[h,t],Generation0, SOC0, batt0, Retailer_VC[k,t])
            loads0.append(instance0.Pgrid)
        column_name = f'House {h}'
        CustomerLoad0[column_name] = loads0
        bill0 = Bill(CustomerLoad0.iloc[:,h],Retailer_VC[k],Retailer_FC[k],T_cost[k],D_cost[k],M_cost[k],D_Fcost[k],h)
        Bill0[k,h] = bill0.electricity_bill

        systemsize=Systemsize(Pd.iloc[h,:].sum())
        options=len(systemsize.df_combinations)
        
        #OFFGRID OPERATION
        SOC=SOC_max #fully charged battery
        PVSizeOff=systemsize.df_combinations.iloc[0,0]+PV[h]
        BattSizeOff=systemsize.df_combinations.iloc[0,1]+Battery[h]
        SystemCostOff=systemsize.df_combinations.iloc[0,2]
        genoff=[]
        curtailoff=[]
        battoff=[]
        CustomerSOCOff[h,0]=SOC_max if BattSizeOff>0 else 0
        SOC=CustomerSOCOff[h,0]
        for t in range(0,hours):
            instanceOff = CustomerOffgrid(Pd.iloc[h,t], PVSizeOff, SOC, BattSizeOff)
            SOC=instanceOff.SOC2 
            CustomerSOCOff[h,t+1]=SOC
            ECurtail[h,t]=instanceOff.Pcurtail
            CustomerbattOff[h,t]=instanceOff.Pb 

        if all(x <= 0 for x in ECurtail[h]):
            loff=LCOEoff(SystemCostOff)
            lcoeoffgrid[h]=loff.lcoeoff
        else:
            lcoeoffgrid[h]=100  
 
        # PV-Batt Combinations
        #ONLY new options at k, no previous PV-Batt
        if flags[h]==False:
            CustomerLoadPVonlyk=np.zeros((hours,options))
            CustomerSOCPVonlyk=np.zeros((hours+1,options))
            CustomerBattPVonlyk=np.zeros((hours,options))
            PVSizeonlyk=np.zeros(options)
            BattSizeonlyk=np.zeros(options)
            SystemCost=np.zeros(options)
            for i in range (0,options):
                PVSizeonlyk[i]=systemsize.df_combinations.iloc[i,0]
                BattSizeonlyk[i]=systemsize.df_combinations.iloc[i,1]
                SystemCost[i]=systemsize.df_combinations.iloc[i,2]
                load3=[]
                gen3=[]
                soc3=[]
                batt3=[]
                CustomerSOCPVonlyk[0,i]=SOC_max if BattSizeonlyk[i]>0 else 0
                SOC=CustomerSOCPVonlyk[0,i]
                for t in range(0,hours):
                    instance3 = Customer(Pd.iloc[h,t], PVSizeonlyk[i], SOC, BattSizeonlyk[i], Retailer_VC[k,t])
                    SOC=instance3.SOC2
                    CustomerLoadPVonlyk[t,i]=instance3.Pgrid
                    CustomerSOCPVonlyk[t+1,i]=instance3.SOC2
                    CustomerBattPVonlyk[t,i]=instance3.Pb
                billonlyk=Bill(CustomerLoadPVonlyk[:,i],Retailer_VC[k],Retailer_FC[k],T_cost[k],D_cost[k],M_cost[k],D_Fcost[k],h)
                Billonlyk[h,i]=billonlyk.electricity_bill

            # PREVIOUS PV-BATT at k-1 and new options at k
            Loadkminus1k=np.zeros((hours,options))
            SOCkminus1k=np.zeros((hours+1,options))
            Battkminus1k=np.zeros((hours,options))
            PVSizekminus1k=np.zeros(options)
            BattSizekminus1k=np.zeros(options)
            for i in range (0,options):
                PVSizekminus1k[i]=(systemsize.df_combinations.iloc[i,0]) + (pvk[k-1,h] if k>=1 else 0)
                BattSizekminus1k[i]=(systemsize.df_combinations.iloc[i,1]) + (battk[k-1,h] if k>=1 else 0)
                load4=[]
                gen4=[]
                soc4=[]
                batt4=[]
                SOCkminus1k[0,i]=SOC_max if BattSizekminus1k[i]>0 else 0
                SOC=SOCkminus1k[0,i]
                for t in range(0,hours):
                    instance4 = Customer(Pd.iloc[h,t], PVSizekminus1k[i], SOC, BattSizekminus1k[i], Retailer_VC[k,t])
                    SOC=instance4.SOC2
                    Loadkminus1k[t,i]=instance4.Pgrid
                    SOCkminus1k[t+1,i]=instance4.SOC2
                    Battkminus1k[t,i]=instance4.Pb
                billkminus1k=Bill(Loadkminus1k[:,i],Retailer_VC[k],Retailer_FC[k],T_cost[k],D_cost[k],M_cost[k],D_Fcost[k],h)
                Billkminus1k[h,i]=billkminus1k.electricity_bill

            # PREVIOUS PV-BATT at k-1, k-2, and new options at k
            CustomerLoadPV=np.zeros((hours,options))
            CustomerSOCPV=np.zeros((hours+1,options))
            CustomerBattPV=np.zeros((hours,options))
            PVSize=np.zeros(options)
            BattSize=np.zeros(options)
            for i in range (0,options):
                PVSize[i]=systemsize.df_combinations.iloc[i,0]+PV[h]
                BattSize[i]=systemsize.df_combinations.iloc[i,1]+Battery[h]
                load2=[]
                gen2=[]
                soc2=[]
                batt2=[]
                CustomerSOCPV[0,i]=SOC_max if BattSize[i]>0 else 0
                SOC=CustomerSOCPV[0,i]
                for t in range(0,hours):
                    instance2 = Customer(Pd.iloc[h,t], PVSize[i], SOC, BattSize[i], Retailer_VC[k,t])
                    SOC=instance2.SOC2
                    CustomerLoadPV[t,i]=instance2.Pgrid
                    CustomerSOCPV[t+1,i]=instance2.SOC2
                    CustomerBattPV[t,i]=instance2.Pb
                bill=Bill(CustomerLoadPV[:,i],Retailer_VC[k],Retailer_FC[k],T_cost[k],D_cost[k],M_cost[k],D_Fcost[k],h)
                BillPVCombinations[h,i]=bill.electricity_bill
                Echarge[h,i]=bill.electricity_charges
                Dcharge[h,i]=bill.delivery_charges

            PVSizedecision=np.append(systemsize.df_combinations.iloc[:,0].values,PVSizeOff)
            BattSizedecision=np.append(systemsize.df_combinations.iloc[:,1], BattSizeOff)
            SystemCost=np.append(SystemCost,SystemCostOff)

            decision=Decision(BillPVCombinations[h],PVSizedecision,BattSizedecision,SystemCost,lcoeoffgrid[h]) #change name to lcoe

            ind=decision.index
            battk[k,h]=decision.SelectedBatt
            pvk[k,h]=decision.Selected_PV
            Inv[k,h]=SystemCost[ind]

            if ind==options:
                hh=hh+1
                flags[h] = True  # Set the flag to True for the houses that go off-grid
                Customerloadactual[:,h]==np.zeros(hours)
            else:
                Customerloadactual[:,h]=CustomerLoadPV[:,ind]
            Customerbattactual[:,h]=CustomerBattPV[:,ind]
            Customersocactual[:,h]=CustomerSOCPV[:,ind]
            Loadkminus1kactual[:,h]=Loadkminus1k[:,ind]
            CustomerLoadPVonlykactual[:,h]=CustomerLoadPVonlyk[:,ind]
            # Billexpected[k,h]=Billkminus1k[h,ind]

    Load2 = np.where(Customerloadactual >= 0, Customerloadactual, 0)
    Inj2 = np.where(Customerloadactual < 0, Customerloadactual, 0)
    TotalLoad2_h = Load2.sum() #kWh
    TotalInj2_h = Inj2.sum() #kWh
    HourlyLoad2_h = (Load2.sum(axis=1)) #kWh
    HourlyInj2_h = (Inj2.sum(axis=1)) #kWh
    HourlyLoad20 = pd.DataFrame(HourlyLoad2_h)
    HourlyInj20 = pd.DataFrame(HourlyInj2_h)
    
    for y in range(yy,yyend):
        ind=y-yy
        Phour2[ind] = HourlyLoad20[0]*households.iloc[0,ind]
    HourlyLoad2=(Phour2)+(base*1000) #kWh
    HourlyResLoad2=Phour2

    for y in range(yy,yyend):
        ind=y-yy
        Pinj2[ind] = HourlyInj20[0]*households.iloc[0,ind]
    HourlyInj2 = (Pinj2) #kWh
    HourlyNet2=HourlyLoad2+HourlyInj2
    
    TotalLoad2=(TotalLoad2_h*households.values.sum())+(base.values.sum()*1000) #kWh
    Totalresload2=(TotalLoad2_h*households.values.sum()) #kWh
    TotalInj2=(TotalInj2_h*households.values.sum()) #kWh
    
    TotalNet2=TotalLoad2+TotalInj2
    
    print('Actual Annual Load:',TotalLoad2, 'kWh')
    print('Actual Res Load:',Totalresload2, 'kWh')
    print('Actual Annual Injection:', TotalInj2, 'kWh')
    
    Distributionactual=Delivery_Costs(Totalresload2,houses-hh)
    Retaileractual=Energy_costs(TotalNet2,cont_vol,cont_price,WS_price,ret_margin_E,ret_margin_NE,risk_margin,admin_ch,fees,HourlyResLoad2,HourlyNet2,HourlyLoad2,HourlyInj2,houses-hh,Distributionactual.dist_var_cost_pu)
    # Distributionactual=Delivery_Costs(TotalLoad2,houses-hh)
    energycostactual[k]=Retaileractual.energy_proc_cost.sum()
    Retailer_VC_actual[k]=Retaileractual.Ret_Var_Cost_pu
    Retailer_FC_actual[k]=Retaileractual.ret_fixed_cost_HH
    Retailer_Rev_actual[k]=Retaileractual.ret_rev
    ugenpaymentactual[k]=Retaileractual.ret_ugen_payment
    # actualprofit[k]=Retailer_Rev_actual[k]+ugenpaymentactual[k]-energycostactual[k]
    Retailer_VC[k+1]=Retaileractual.Ret_Var_Cost_pu
    T_cost_actual[k]=Distributionactual.T_cost
    D_cost_actual[k]=Distributionactual.D_cost
    M_cost_actual[k]=Distributionactual.M_cost
    D_Fcost_actual[k]=Distributionactual.dist_fixed_cost_HH
    DSO_charges_actual[k]=Retaileractual.dsocharges
    print('Actual Total Energy Cost',energycostactual[k] ,'C$')
    print("Actual Retailer VC", Retailer_VC_actual[k], 'C$/kWh')
    print("Actual Retailer FC", Retailer_FC_actual[k], 'C$/h')
    print("Actual Transmission cost", T_cost_actual[k], 'C$/kWh')
    print("Actual Distribution cost", D_cost_actual[k], 'C$/kWh')
    print("Actual Municipal cost", M_cost_actual[k], 'C$/kWh')
    # print('Actual Retailer Revenue', Retailer_Rev_actual[k], 'C$')
    print('Actual Payment by Retailer for Injections', ugenpaymentactual[k], 'C$')
    # print('Actual Retailer Profit', actualprofit[k], 'C$')
    
#     fig = go.Figure()
#     fig.add_traces(go.Scatter(y = HourlyLoad2, mode = 'lines', line=dict(color=colors[0]), name='Load'))
#     fig.add_traces(go.Scatter(y = Retailer_VC_actual[k], mode = 'lines', line=dict(color=colors[1]), name='Retail Price'))
#     fig.update_layout(title='Actual')
    
#         # pio.write_html(fig, 'Offgrid.html')
    
# BILL WITH investment at k-2,k-1,k
    for h in range(0,houses):
        if flags[h]==False:
            bill_actual=Bill(Customerloadactual[:,h],Retailer_VC_actual[k],Retailer_FC_actual[k],T_cost_actual[k],D_cost_actual[k],M_cost_actual[k],D_Fcost_actual[k],h)
            Actualdcharge[k,h]=bill_actual.delivery_charges #total bill of each house
            Actualbill[k,h]=bill_actual.electricity_bill #total bill of each house
            print('credits of house', h, bill_actual.energy_credits)

            bill_actual_kminus1k=Bill(Loadkminus1kactual[:,h],Retailer_VC_actual[k],Retailer_FC_actual[k],T_cost_actual[k],D_cost_actual[k],M_cost_actual[k],D_Fcost_actual[k],h)
            Actualbillkminus1k[k,h]=bill_actual_kminus1k.electricity_bill

            bill_actual_onlyk=Bill(CustomerLoadPVonlykactual[:,h],Retailer_VC_actual[k],Retailer_FC_actual[k],T_cost_actual[k],D_cost_actual[k],M_cost_actual[k],D_Fcost_actual[k],h)
            Actualbillonlyk[k,h]=bill_actual_onlyk.electricity_bill       

            bill0actual = Bill(CustomerLoad0.iloc[:,h],Retailer_VC_actual[k],Retailer_FC_actual[k],T_cost_actual[k],D_cost_actual[k],M_cost_actual[k],D_Fcost_actual[k],h)
            Actualbill0[k,h] = bill0actual.electricity_bill
            
    Retailer_Revenue_Household_actual[k]=Actualbill[k].sum()*(households.sum(axis=1))
    actualprofit[k]=Retailer_Revenue_Household_actual[k]+Retailer_Rev_actual[k]-energycostactual[k]-DSO_charges_actual[k]+ugenpaymentactual[k]-admin_ch
    print("Retailer revenue total", Retailer_Revenue_Household_actual[k]+Retailer_Rev_actual[k], 'C$')
    print('Cost to DSO', DSO_charges_actual[k], 'C$')
    print('Retailer Profit', actualprofit[k], 'C$')
            
    dfpv = pd.DataFrame({f'PV{j+1}': pvk[:, j] for j in range(pvk.shape[1])})
    dfb = pd.DataFrame({f'Batt{j+1}': battk[:, j] for j in range(battk.shape[1])})
    print(dfpv)
    print(dfb)
    print("-------------------------------")
    yy=yy+5
    yyend=yyend+5

k= 0
TotalLoad_h [37714.60083216]
households.values.sum() 5566761.281377016
(base.values.sum()*1000) 669202285712.9642
Reductions in houseshold load [0.]
Increased base load 71781081763.25415
Total Grid Load for 15 years: [8.79150465e+11] kWh
Total Res Load for 15 years: [2.0994818e+11] kWh
Total Injection for 15 years: 0.0 kWh
market cost [1.7442721e+10]
cont cost 93543179220.0
EP cost [1.3318308e+11]
load [8.79150465e+11]


C:\Users\ASUS\AppData\Local\Temp\ipykernel_14388\127445526.py:127: FutureWarning:

Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead



Total Energy Cost 110985900203.14035 C$
Retailer VC [0.14180014 0.13919325 0.13723829 ... 0.14062304 0.14044411 0.14089675] C$/kWh
Retailer FC 0.009077153506482069 C$/h
Transmission cost 0.042094 C$/kWh
Distribution cost 0.013033 C$/kWh
Municipal cost 0.017162 C$/kWh
Payment by Retailer for Injections 0.0 C$
Retailer revenue total 156282846015.98492 C$
Cost to DSO 20886283935.679928 C$
Retailer Profit 22302824558.304646 C$


C:\Users\ASUS\AppData\Local\Temp\ipykernel_14388\127445526.py:158: FutureWarning:

Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead



    PVSize  BattSize                                 Total_Cost
0      3.2       3.0   0    12736.12956
Name: 0, dtype: float64
1      3.2       1.0  0    11989.655682
Name: 0, dtype: float64
2      3.2       0.0  0    11616.418743
Name: 0, dtype: float64
3      3.2       0.0  0    11616.418743
Name: 0, dtype: float64
4      1.6       3.0   0    6927.920188
Name: 0, dtype: float64
5      1.6       1.0   0    6181.446311
Name: 0, dtype: float64
6      1.6       0.0   0    5808.209372
Name: 0, dtype: float64
7      1.6       0.0   0    5808.209372
Name: 0, dtype: float64
8      0.8       3.0   0    4023.815503
Name: 0, dtype: float64
9      0.8       1.0   0    3277.341625
Name: 0, dtype: float64
10     0.8       0.0   0    2904.104686
Name: 0, dtype: float64
11     0.8       0.0   0    2904.104686
Name: 0, dtype: float64
12     0.0       3.0   0    1119.710817
Name: 0, dtype: float64
13     0.0       1.0    0    373.236939
Name: 0, dtype: float64
14     0.0       0.0           0    0.0


...

C:\Users\ASUS\AppData\Local\Temp\ipykernel_14388\127445526.py:218: FutureWarning:

Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead



grid lcoe 0.23
LCOE combinations [0.231 0.231 0.231 0.231 0.231 0.231 0.231 0.231 0.231 0.231 0.231 0.231
 0.231 0.231 0.231 0.231]
No index found for selected LCOE.
Actual Annual Load: 934156106584.7244 kWh
Actual Res Load: 149833929371.97675 kWh
Actual Annual Injection: -162210700473.36514 kWh
market cost 14553752041.131014
cont cost 93543179220.0
EP cost 129716317513.35721
load 771945406111.3595
Actual Total Energy Cost 108096931261.13101 C$
Actual Retailer VC [0.1272417  0.12370524 0.12101923 ... 0.12578811 0.12560807 0.12627862] C$/kWh
Actual Retailer FC 0.00744642744674837 C$/h
Actual Transmission cost 0.042094 C$/kWh
Actual Distribution cost 0.013033 C$/kWh
Actual Municipal cost 0.017162 C$/kWh
Actual Payment by Retailer for Injections -22919876081.931927 C$
credits of house 0 327.3622625896989
credits of house 1 690.4545661317404
credits of house 2 858.5297587604517
credits of house 3 1035.1469521263477


C:\Users\ASUS\AppData\Local\Temp\ipykernel_14388\127445526.py:343: FutureWarning:

Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead



credits of house 4 466.1043897412805
Retailer revenue total 125550608164.92682 C$
Cost to DSO 16540684896.965103 C$
Retailer Profit -24114721393.961224 C$
   PV1  PV2  PV3  PV4  PV5
0  0.8  1.6  2.0  2.4  1.2
1  2.4  5.2  6.0  8.0  3.6
2  0.0  0.0  0.0  0.0  0.0
   Batt1  Batt2  Batt3  Batt4  Batt5
0    0.0    0.0    0.0    0.0    0.0
1    0.0    0.0    0.0    0.0    0.0
2    0.0    0.0    0.0    0.0    0.0
-------------------------------


C:\Users\ASUS\AppData\Local\Temp\ipykernel_14388\127445526.py:387: FutureWarning:

Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead



In [65]:
actualprofit/(Retailer_Revenue_Household_actual+Retailer_Rev_actual)

array([ 0.13335753, -0.20341836, -0.19207172])

In [66]:
Customerloadactual.sum()

-1823.908484262568

In [67]:
Actualbill.sum(axis=0)*5/(15*12)

array([ 41.72767985,  67.7398575 ,  82.61965367, 100.67194223,
        52.03807814])

In [68]:
Load2[:,0].sum()
MonthInv=np.zeros((1,5))
MonthInv2=Inv
for h in range(0,5):
    MonthInv[0,h]=(Inv[0,h]+Inv[1,h]+Inv[2,h])/(15*12)
print(MonthInv)

[[ 45.70457506  96.33759348 114.26143766 146.9706119   68.55686259]]


In [69]:
Actualdcharge.sum(axis=0)*5/(15*12)

array([28.68988034, 44.48823239, 52.82021219, 61.81603567, 34.79729904])

In [70]:
Inj2[:,0].sum()

-2315.197870883776

In [71]:
(Pd.iloc[3,:]).sum()

12059.624404801998

In [72]:
(Pg[0]*10.4).sum()

12129.4992

In [73]:
(Pd.iloc[0,:]).sum()-(Pg[0]*0.8).sum()

2568.03818491

In [74]:
Customerloadactual[:,4].sum()

-426.45837426736205

In [75]:
((Phour2)+(base*1000)).sum()

0     5.534375e+10
1     5.626390e+10
2     5.719947e+10
3     5.815072e+10
4     5.911791e+10
5     6.010131e+10
6     6.110120e+10
7     6.211785e+10
8     6.315154e+10
9     6.420257e+10
10    6.527123e+10
11    6.635782e+10
12    6.746263e+10
13    6.858599e+10
14    6.972820e+10
dtype: float64

In [76]:
((Phour2)).sum()

0     8.664218e+09
1     8.837502e+09
2     9.014252e+09
3     9.194537e+09
4     9.378428e+09
5     9.565996e+09
6     9.757316e+09
7     9.952463e+09
8     1.015151e+10
9     1.035454e+10
10    1.056163e+10
11    1.077287e+10
12    1.098832e+10
13    1.120809e+10
14    1.143225e+10
dtype: float64

In [77]:
Pinj2.sum()

0    -9.379910e+09
1    -9.567509e+09
2    -9.758859e+09
3    -9.954036e+09
4    -1.015312e+10
5    -1.035618e+10
6    -1.056330e+10
7    -1.077457e+10
8    -1.099006e+10
9    -1.120986e+10
10   -1.143406e+10
11   -1.166274e+10
12   -1.189599e+10
13   -1.213391e+10
14   -1.237659e+10
dtype: float64

In [78]:
((Phour2)+(base*1000)).sum()+Pinj2.sum()

0     4.596384e+10
1     4.669639e+10
2     4.744062e+10
3     4.819669e+10
4     4.896480e+10
5     4.974513e+10
6     5.053790e+10
7     5.134328e+10
8     5.216148e+10
9     5.299271e+10
10    5.383717e+10
11    5.469508e+10
12    5.556664e+10
13    5.645208e+10
14    5.735161e+10
dtype: float64

In [79]:
# # simple_array = [element[0] for element in HourlyLoad1]
# import datetime
colors = px.colors.qualitative.Plotly
fig = go.Figure()
# # start_date = datetime.datetime(2012, 1, 1, 23)
# # end_date = datetime.datetime(2022, 12, 31, 23)
# # date_rng = pd.date_range(start=start_date, end=end_date, freq='H')
# # timestamps = [datetime.datetime(2000, 1, 1, i) for i in range(0,23) ] * 365 * 20  # Repeated for 20 years

# fig.add_traces(go.Scatter(y = Load2[:,0], mode = 'lines', line=dict(color=colors[3]), name='Load'))
# fig.add_traces(go.Scatter(y = Inj2[:,0], mode = 'lines', line=dict(color=colors[4]), name='cost'))
# fig.add_traces(go.Scatter(y = HourlyNet2.sum(axis=1), mode = 'lines', line=dict(color=colors[2]), name='MSP'))
# fig.add_traces(go.Scatter(y = Retaileractual.energy_proc_cost, mode = 'lines', line=dict(color=colors[1]), name='Retail Price Actual k = 0'))
# fig.add_traces(go.Scatter(y = Customerloadactual[:,0], mode = 'lines', line=dict(color=colors[0]), name='Retail Price Base'))
# fig.add_traces(go.Scatter(y = Retaileractual.energy_proc_cost/HourlyNet2.sum(axis=1), mode = 'lines', line=dict(color=colors[0]), name='Retail Price Actual k = 1'))
# fig.add_traces(go.Scatter(y = Retailer_VC_actual[2]/10, mode = 'lines', line=dict(color=colors[3]), name='Retail Price Actual k = 2'))
# fig.add_traces(go.Scatter(y = Phour2, mode = 'lines', line=dict(color=colors[0]), name='Retail Price Base'))
# fig.add_traces(go.Scatter(y = base*1000, mode = 'lines', line=dict(color=colors[1]), name='Retail Price Base'))
fig.add_traces(go.Scatter(y = Pd.iloc[3], mode = 'lines', line=dict(color=colors[2]), name='Load'))
fig.add_traces(go.Scatter(y = Pg[0]*10.4, mode = 'lines', line=dict(color=colors[3]), name='PV'))
fig.add_traces(go.Scatter(y = Pd.iloc[3]-Pg[0]*10.4, mode = 'lines', line=dict(color=colors[4]), name='Retail Price Base'))


# fig.add_traces(go.Scatter(y = Retailer_VC_actual[0], mode = 'lines', line=dict(color=colors[0]), name='Retail Price Actual'))

# fig.update_layout(title='Market Settlement Price ($/MWh)')
# fig.update_xaxes(title_text='Date', tickformat='%Y-%m-%d')
# fig.update_yaxes(title_text="Price") # Y-axis label
# # pio.write_html(fig, 'Prices2.html')
# # fig.show()
              


In [80]:
TotalInj2

-162210700473.36514

In [81]:
Customerloadactual[5388,:]

array([-1.96058601, -4.18217886, -5.37563363, -6.98949558, -2.62642928])

In [82]:
Phour2.iloc[5388,:]

0     0.0
1     0.0
2     0.0
3     0.0
4     0.0
5     0.0
6     0.0
7     0.0
8     0.0
9     0.0
10    0.0
11    0.0
12    0.0
13    0.0
14    0.0
Name: 5388, dtype: float64

In [83]:
HourlyNet2.iloc[5388,:]

0    -1.512241e+06
1    -1.569609e+06
2    -1.628558e+06
3    -1.689128e+06
4    -1.751356e+06
5    -1.815284e+06
6    -1.880953e+06
7    -1.948406e+06
8    -2.017684e+06
9    -2.088834e+06
10   -2.161899e+06
11   -2.236925e+06
12   -2.313961e+06
13   -2.393055e+06
14   -2.474255e+06
Name: 5388, dtype: float64

In [84]:
HourlyLoad20.iloc[5388,0]

0.0

In [85]:
base.iloc[5388,:]*1000

0     6.780762e+06
1     6.889254e+06
2     6.999482e+06
3     7.111474e+06
4     7.225258e+06
5     7.340862e+06
6     7.458316e+06
7     7.577649e+06
8     7.698891e+06
9     7.822073e+06
10    7.947227e+06
11    8.074382e+06
12    8.203572e+06
13    8.334829e+06
14    8.468187e+06
Name: 5388, dtype: float64

In [86]:
 Pinj2.iloc[5388,:]

0    -8.293004e+06
1    -8.458864e+06
2    -8.628041e+06
3    -8.800602e+06
4    -8.976614e+06
5    -9.156146e+06
6    -9.339269e+06
7    -9.526054e+06
8    -9.716575e+06
9    -9.910907e+06
10   -1.010913e+07
11   -1.031131e+07
12   -1.051753e+07
13   -1.072788e+07
14   -1.094244e+07
Name: 5388, dtype: float64

In [87]:
(Pg[0]*8).sum()

9330.384

In [88]:
Retaileractual.energy_proc_cost

0       1.083921e+07
1       1.021017e+07
2       9.832091e+06
3       9.643418e+06
4       9.668008e+06
            ...     
8755    1.265268e+07
8756    1.241094e+07
8757    1.254140e+07
8758    1.245141e+07
8759    1.182336e+07
Length: 8760, dtype: float64

In [89]:
WS_price

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.01735,0.02831,0.02165,0.14140,0.02450,0.01019,0.01060,0.05202,0.05454,0.03754,0.05292,0.03928,0.05112,0.03112,0.03660
1,0.01200,0.01768,0.02100,0.14141,0.02263,0.01488,0.01060,0.01084,0.04002,0.03568,0.03497,0.02627,0.04879,0.03077,0.03176
2,0.00975,0.01199,0.00590,0.14140,0.01700,0.01019,0.01060,0.01084,0.03565,0.03306,0.01758,0.01702,0.03956,0.02713,0.02974
3,0.00544,0.01199,0.00585,0.14137,0.01700,0.01019,0.01060,0.01084,0.01095,0.03219,0.02006,0.01701,0.03627,0.02411,0.02486
4,0.00544,0.01199,0.00584,0.14136,0.01700,0.01019,0.01060,0.01033,0.01095,0.03206,0.02235,0.01700,0.03616,0.02408,0.02401
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,0.03264,0.03173,0.14506,0.03766,0.03008,0.02427,0.04503,0.07050,0.04993,0.04792,0.02800,0.04897,0.02363,0.02872,0.01932
8756,0.03264,0.03173,0.14498,0.03626,0.01639,0.01063,0.02928,0.05927,0.04890,0.04355,0.02812,0.04913,0.02384,0.02291,0.01923
8757,0.03060,0.03199,0.14504,0.02939,0.01013,0.02506,0.02465,0.05555,0.04993,0.04307,0.03941,0.04909,0.02410,0.02425,0.01922
8758,0.03004,0.03200,0.14509,0.02560,0.01013,0.02274,0.01125,0.05089,0.04993,0.04476,0.03766,0.04756,0.02428,0.03095,0.01890


In [90]:
HourlyNet2

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,6.032360e+06,6.135929e+06,6.241296e+06,6.348493e+06,6.457551e+06,6.568505e+06,6.681386e+06,6.796229e+06,6.913068e+06,7.031939e+06,7.152876e+06,7.275918e+06,7.401100e+06,7.528460e+06,7.658037e+06
1,5.847401e+06,5.947426e+06,6.049180e+06,6.152694e+06,6.257998e+06,6.365125e+06,6.474106e+06,6.584973e+06,6.697760e+06,6.812500e+06,6.929227e+06,7.047976e+06,7.168784e+06,7.291684e+06,7.416715e+06
2,5.755487e+06,5.853989e+06,5.954196e+06,6.056136e+06,6.159841e+06,6.265342e+06,6.372669e+06,6.481855e+06,6.592933e+06,6.705935e+06,6.820896e+06,6.937849e+06,7.056830e+06,7.177874e+06,7.301018e+06
3,5.707890e+06,5.805678e+06,5.905159e+06,6.006365e+06,6.109323e+06,6.214067e+06,6.320626e+06,6.429033e+06,6.539320e+06,6.651520e+06,6.765666e+06,6.881794e+06,6.999936e+06,7.120130e+06,7.242411e+06
4,5.722245e+06,5.820414e+06,5.920287e+06,6.021892e+06,6.125261e+06,6.230423e+06,6.337412e+06,6.446259e+06,6.556995e+06,6.669656e+06,6.784274e+06,6.900885e+06,7.019522e+06,7.140222e+06,7.263020e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,7.010211e+06,7.134332e+06,7.260678e+06,7.389290e+06,7.520208e+06,7.653475e+06,7.789133e+06,7.927225e+06,8.067796e+06,8.210891e+06,8.356556e+06,8.504837e+06,8.655782e+06,8.809440e+06,8.965860e+06
8756,6.989463e+06,7.112680e+06,7.238096e+06,7.365751e+06,7.495685e+06,7.627940e+06,7.762558e+06,7.899581e+06,8.039052e+06,8.181017e+06,8.325520e+06,8.472607e+06,8.622325e+06,8.774722e+06,8.929846e+06
8757,7.044555e+06,7.167680e+06,7.292984e+06,7.420505e+06,7.550284e+06,7.682359e+06,7.816774e+06,7.953568e+06,8.092787e+06,8.234471e+06,8.378667e+06,8.525419e+06,8.674773e+06,8.826775e+06,8.981473e+06
8758,7.017221e+06,7.138045e+06,7.260974e+06,7.386043e+06,7.513292e+06,7.642758e+06,7.774480e+06,7.908499e+06,8.044855e+06,8.183588e+06,8.324742e+06,8.468359e+06,8.614481e+06,8.763155e+06,8.914424e+06


In [91]:
np.average(Retailer_VC_actual[0])

0.1534755208789863

In [92]:
np.average(Retailer_VC_actual[1])

0.14647293420471244

In [93]:
np.average(Retailer_VC_actual[2])

0.138902279813407

In [94]:
np.average(Retailer_VC_actual[0:2])

0.1499742275418494

In [95]:
Retailer_VC_actual

array([[0.14199332, 0.13919325, 0.13723829, ..., 0.14070827, 0.14049485,
        0.14098413],
       [0.13421463, 0.1311245 , 0.12878853, ..., 0.13288823, 0.13271316,
        0.13328216],
       [0.1272417 , 0.12370524, 0.12101923, ..., 0.12578811, 0.12560807,
        0.12627862],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [96]:
solar_prod_annual

1167

In [97]:
Pd.iloc[4,:].sum()

5291.0723057840005

In [98]:
(WS_price*np.maximum(0,HourlyLoad1-cont_vol)).values.sum()

array([1.92007329e+10])

In [99]:
households*5

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,1.961975e+06,2.001215e+06,2.041239e+06,2.082064e+06,2.123705e+06,2.166179e+06,2.209503e+06,2.253693e+06,2.298767e+06,2.344742e+06,2.391637e+06,2.439470e+06,2.488259e+06,2.538024e+06,2.588785e+06


In [100]:
csv_file_path = 'O3.csv'  # Specify the file path where you want to save the CSV file
a.to_csv(csv_file_path, index=False)  # Set index=False to exclude the DataFrame index from the CSV file

In [101]:
# csv_file = 'output1.csv'

# # Write the array into the CSV file
# with open(csv_file, 'w', newline='') as file:
#     writer = csv.writer(file)
#     writer.writerows(a)

# print("Array written to", csv_file)

In [102]:
households*5

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,1.961975e+06,2.001215e+06,2.041239e+06,2.082064e+06,2.123705e+06,2.166179e+06,2.209503e+06,2.253693e+06,2.298767e+06,2.344742e+06,2.391637e+06,2.439470e+06,2.488259e+06,2.538024e+06,2.588785e+06


In [103]:
for y in range(0,25):
    print(1002007*(1+0.02)**(y))

1002007.0
1022047.14
1042488.0828
1063337.8444560003
1084604.60134512
1106296.6933720224
1128422.627239463
1150991.0797842522
1174010.9013799373
1197491.119407536
1221440.9417956867
1245869.7606316006
1270787.1558442325
1296202.8989611173
1322126.9569403396
1348569.4960791464
1375540.8860007294
1403051.703720744
1431112.737795159
1459734.992551062
1488929.6924020834
1518708.286250125
1549082.4519751277
1580064.1010146302
1611665.3830349229


In [104]:
for y in range(0,25):
    print(totalhouses*5*(1+0.022)**(y))

1609503.0
1644912.066
1681100.131452
1718084.334343944
1755882.1896995108
1794511.5978729
1833990.853026104
1874338.6517926785
1915574.1021321171
1957716.7323790241
2000786.5004913625
2044803.8035021725
2089789.4871792202
2135764.855897163
2182751.6827269006
2230772.219746893
2279849.2085813247
2330005.891170114
2381266.020775856
2433653.873232925
2487194.2584440494
2541912.5321298186
2597834.6078366744
2654986.9692090815
2713396.6825316814
